In [123]:
import numpy as np
import pandas as pd

In [99]:
topic_action_df = pd.read_csv('topic_actions_list.csv')

In [100]:
topic_action_df['CleanedAction'] = topic_action_df['Asked info or doc'].apply(lambda x: str(x).lower())

In [101]:
index_to_action_df = pd.read_csv('index_to_action_lookup.csv')

In [102]:
merged_df = pd.merge(topic_action_df, index_to_action_df, left_on='CleanedAction', right_on='Action')

In [103]:
topic_action_new = pd.DataFrame(merged_df.loc[:, ['Topic', 'Action ID']].groupby('Topic')['Action ID'].apply(list)).reset_index()
topic_action_new = topic_action_new[topic_action_new['Topic'] > -1]
topic_action_new.reset_index(drop=True, inplace=True)
topic_action_new.rename(columns={'Action ID': 'ActionIDs_adhoc'}, inplace=True)

In [104]:
action_representation_map = pd.read_csv('./action_representation_map.csv')

In [105]:
topic_action_model = pd.DataFrame(action_representation_map.loc[:, ['TopicNumber', 'ActionID']].groupby('TopicNumber')['ActionID'].apply(list)).reset_index()

In [106]:
topic_action_model.to_csv('topic_action_model.csv')
topic_action_new.to_csv('topic_action_new.csv')

In [107]:
topic_action_model.rename(columns={'ActionID': 'ActionIDs_model'}, inplace=True)

In [108]:
topic_action_model

,TopicNumber,ActionIDs_model
0,0,[26]
1,1,[8]
2,2,"[15, 55]"
3,3,"[4, 30, 56]"
4,4,"[57, 58]"
...,...,...
72,79,"[0, 32]"
73,80,[16]
74,81,[11]
75,82,[3]


In [109]:
comparison_df = pd.merge(topic_action_new, topic_action_model, left_on='Topic', right_on='TopicNumber').drop(columns=['TopicNumber'])

In [110]:
comparison_df['len_adhoc_actions'] = comparison_df.apply(lambda x: len(set(x['ActionIDs_adhoc'])), axis=1)
comparison_df['len_model_actions'] = comparison_df.apply(lambda x: len(set(x['ActionIDs_model'])), axis=1)
comparison_df['model_is_subset_adhoc'] = comparison_df.apply(lambda x: set(x['ActionIDs_model']).issubset(set(x['ActionIDs_adhoc'])), axis=1)
comparison_df['actions_adhoc_minus_model'] = comparison_df.apply(lambda x: set(x['ActionIDs_adhoc']).difference(set(x['ActionIDs_model'])), axis=1)

In [111]:
comparison_df['len_model_actions'].sum()

126

In [112]:
comparison_df.to_csv('topic_wise_action_adhoc_vs_model.csv', index=False)

In [190]:
nactions = 78
ntopics = 84
action_topic_probability_adhoc = np.zeros([nactions, ntopics])
for i in range(len(comparison_df)):
    topic_id = comparison_df.iloc[i]['Topic']
    actions_id_set = comparison_df.iloc[i]['actions_adhoc_minus_model']
    print(topic_id, actions_id_set)
    for i in actions_id_set:
        action_topic_probability_adhoc[i, topic_id] = 1.0

action_topic_probability_adhoc_sum = action_topic_probability_adhoc.sum(axis=0)
action_topic_probability_adhoc_sum[action_topic_probability_adhoc_sum < 0.1e-5] = 1.0
action_topic_probability_adhoc = action_topic_probability_adhoc / action_topic_probability_adhoc_sum[None, :]

0 set()
1 set()
2 {8, 26, 14}
3 {8}
4 set()
5 set()
6 {8}
7 {0, 1, 3, 5}
8 set()
9 set()
10 set()
11 {62}
12 {0, 1, 2, 4, 29, 63}
13 set()
14 set()
15 set()
16 {9}
17 set()
18 set()
19 {8}
20 {0, 1, 2, 65, 68, 5, 6}
22 {6}
23 {10, 3, 30}
24 {69}
25 set()
26 {72, 10, 52}
27 set()
28 set()
29 set()
30 {20}
31 {76}
32 {9}
33 set()
34 set()
35 set()
36 {4, 12}
37 set()
38 set()
39 {18, 4, 6}
40 {3}
41 set()
43 {3, 4, 5, 6}
44 set()
46 set()
47 set()
48 set()
49 {3}
51 set()
52 {0, 1, 2, 4, 5, 6, 12}
53 {2, 4, 6}
55 set()
56 {4, 45, 6}
57 {0, 4, 20}
59 {0, 2, 4}
61 set()
62 {3}
63 {0, 1, 2, 5, 47, 49}
64 {2, 6}
65 {8, 9}
66 set()
67 {3}
68 {0, 2}
69 set()
70 {1, 2, 4, 5, 6}
71 {0, 1, 4, 5, 6}
72 {1, 2, 4, 5, 6}
73 set()
74 {0, 2, 5, 6, 24}
75 set()
76 set()
77 {1, 2, 3, 4, 5, 6, 25}
78 set()
79 {1, 2, 5, 6}
80 set()
81 set()
82 set()
83 set()


In [191]:
action_topic_probability_adhoc.sum()

37.99999999999999

In [200]:
action_topic_probability_adhoc.sum(axis=0)

array([0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.])

In [208]:
comparison_df['actions_adhoc_minus_model'].apply(lambda x: len(x) > 0).sum()

38

In [209]:
np.save('action_probability_adhoc.npy', action_topic_probability_adhoc)